# AI-Powered Health Data Masking Samples

## Initial Setup

In [ ]:
import boto3
import json
client = boto3.client('apigateway')

# Replace these values with the API and Resource IDs from your CloudFormation Output
api_id = '2cbk50ek5l'
images_mask_resource_id = 'smaotd'
images_phi_resource_id = 'g2v7mb'
images_resource_id = 'm8zsye'
images_text_resource_id = '8g102z'
text_mask_resource_id = 'i5tmxg'
text_phi_resource_id = 'mkdva7'
ImageBucketName = 'aipowered-imagebucket-1he2rguqes4fk'

## Text Masking

In [ ]:
# Define and print our initial text payload
payload = {"text": "PERSON INFORMATION\nName: SALAZAR, CARLOS\nMRN: RQ36114734\nED Arrival Time: 11/12/2011 18:15\nSex: Male\nDOB: 2/11/1961"}
print(payload['text']+'\n'+'\n')

### Get PHI locations in our text
/text/phi

In [ ]:
response = client.test_invoke_method(
	restApiId=api_id,
	resourceId=text_phi_resource_id,
	httpMethod='POST',
	headers={"Content-Type": "application/json"},
	body=json.dumps(payload)
)
print(type(response))
out = json.loads(response['body'])
print(payload['text']+'\n'+'\n')
print(json.dumps(out, indent=4, sort_keys=True))

### Mask PHI in our text
/text/mask

In [ ]:
response = client.test_invoke_method(
	restApiId=api_id,
	resourceId=text_mask_resource_id,
	httpMethod='POST',
	headers={"Content-Type": "application/json"},
	body=json.dumps(payload)
)

out = json.loads(response['body'])
print(out["maskedText"])

# Image Masking

In [ ]:
# These values define our sample image and where we want the masked copy stored
# To view this image, browse to the S3 location listed below in the
# AWS Management Console and 'Download' the image from S3
s3_key = 'sample.png'
destination_key = 'masked/' + s3_key

## Get text locations in an image
/images/text

In [ ]:
queryString='/images/text?s3Bucket='+ImageBucketName+'&s3Key='+s3_key
response = client.test_invoke_method(
	restApiId=api_id,
	resourceId=images_text_resource_id,
	httpMethod='GET',
	headers={"Content-Type": "application/json"},
	pathWithQueryString=queryString
)

out = json.loads(response['body'])
print(json.dumps(out, indent=4, sort_keys=True))

## Get PHI locations in an image
/images/phi

In [ ]:
queryString='/images/phi?s3Bucket='+ImageBucketName+'&s3Key='+s3_key
response = client.test_invoke_method(
	restApiId=api_id,
	resourceId=images_phi_resource_id,
	httpMethod='GET',
	headers={"Content-Type": "application/json"},
	pathWithQueryString=queryString
)

out = json.loads(response['body'])
print(json.dumps(out, indent=4, sort_keys=True))

## Mask Image PHI
/images/mask

In [ ]:
payload = {"s3Bucket": ImageBucketName, "s3Key": s3_key, "destinationBucket": ImageBucketName, "destinationKey": destination_key}
response = client.test_invoke_method(
	restApiId=api_id,
	resourceId=images_mask_resource_id,
	httpMethod='POST',
	headers={"Content-Type": "application/json"},
	body=json.dumps(payload)
)

print(response['body'])

# To view this masked image, browse to the S3 location listed below in the
# AWS Management Console and 'Download' the image from S3